In [65]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib
import copy
import pickle
from sklearn.feature_selection import mutual_info_classif
from collections import Counter
#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize


#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel


import spacy
import time 
from spacy_langdetect import LanguageDetector
from owlready2 import *
import json 
from spotlight import *
import nltk
from nltk.corpus import wordnet as wn



warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp

import vectorizer 
importlib.reload(vectorizer)
import rulesets

importlib.reload(rulesets)

from vectorizer import  WeightedTfidfVectorizer
from rulesets import *

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from owlready2 import *

from mdutils.mdutils import MdUtils
from mdutils import Html

import rdflib
import joblib
import h2o

# Carga de datos

In [66]:

dtm_scaled = pd.read_csv('../data/interim/20newsgroup/weighteddtmscaled.csv',sep=',')
del dtm_scaled['Unnamed: 0']
dtm_scaled.head()
dtm_scaled = dtm_scaled.astype('float32')
cut_labels_4 = [1,2,3,4,5,6,7,8,9,10]
for column in dtm_scaled.columns:
    if column=='target':
        continue
    dtm_scaled[column]  = pd.cut(dtm_scaled[column], bins=10, labels=cut_labels_4)

dtm_scaled.head()


,read,24,service,numb,ask,dept,question,news,support,17,...,great,lot,talk,little,happen,claim,doesn,opinion,leave,target
0,1,1,1,1,1,1,1,1,1,1,...,1,6,1,1,1,1,1,1,1,1.0
1,1,1,1,1,1,1,1,1,3,1,...,1,1,1,1,1,1,1,1,1,1.0
2,1,1,1,1,1,2,1,1,1,1,...,4,1,3,1,1,1,1,1,1,0.0
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
4,1,1,1,1,1,1,2,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0


In [67]:
# Topics e importancia de variables

In [68]:
## 1 Experimento: Número de reglas: 10, usando un máximo de 10 términos, sin selección de vocabulario
pds = copy.deepcopy(dtm_scaled)

In [69]:
rs = RuleSet()
rs.target='target'
rs.pds=pds
rs.pds_original = pds 
explainer = Explainer(rs,1,10,10,200)
rs, pdsb = explainer.explain()

*****************************Comienza la genreación del ruleset************************************************
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: window
Igualdad generada: window==6POSITIVOS 139.0NEGATIVOS: 0
Igualdad de la regla window==6
-----------------> COUNTER USADOS
Counter({'window': 1})
|2 - Regla generada: |window==6 => p(0.0), conf(0.011506622516556292) 
ruleset query(window==6) 
	 |2.1 -Quedan | 11941 positivos, generadas 2 reglas
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: window
Igualdad generada: window==9POSITIVOS 25.0NEGATIVOS: 0
Igualdad de la regla window==9
-----------------> COUNTER USADOS
Counter({'window': 2})
|2 - Regla generada: |window==9 => p(0.0), conf(0.0020695364238410598) 
ruleset query(window==6) or (window==9) 
	 |2.1 -Quedan | 11916 positivos, generadas 3 reglas
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: thank
Igualdad generada: thank==8POSITIVOS 1.0NEG

In [70]:
rso = copy.deepcopy(rs)
for key in rso.dictionary.keys():
    print(rso.dictionary.get(key).toString())

window==6 => p(0.0), conf(0.011506622516556292)
window==9 => p(0.0), conf(0.0020695364238410598)
thank==8 => p(0.0), conf(8.278145695364238e-05)
thank==10 => p(0.0), conf(8.278145695364238e-05)
thank==9 => p(0.0), conf(8.278145695364238e-05)
thank==4 and window==7 => p(0.0), conf(8.278145695364238e-05)
thank==4 and window==5 => p(0.0), conf(0.0006622516556291391)
thank==4 and window==2 => p(0.0), conf(8.278145695364238e-05)
thank==4 and window==3 => p(0.0), conf(0.00024834437086092715)
thank==4 and window==1 and claim==5 => p(0.0), conf(8.278145695364238e-05)


In [72]:
consolidar = rso.consolidar()

0
diccionary
{0: <rulesets.Rule object at 0x7f14100ea3d0>, 1: <rulesets.Rule object at 0x7f14100eaf10>, 2: <rulesets.Rule object at 0x7f14100ea050>, 3: <rulesets.Rule object at 0x7f14100eab90>, 4: <rulesets.Rule object at 0x7f14100ea690>, 5: <rulesets.Rule object at 0x7f14100eac90>, 6: <rulesets.Rule object at 0x7f14549b7190>, 7: <rulesets.Rule object at 0x7f1448e1efd0>, 8: <rulesets.Rule object at 0x7f131171c510>, 9: <rulesets.Rule object at 0x7f131171ca50>}
visitados q no deben volver :0 -- 0 -- 1
listado de reglas con mismo antecedente:
window==6 => p(0.0), conf(0.011506622516556292)
window==9 => p(0.0), conf(0.0020695364238410598)
listado_vari::
window
  window  confianza  consecuente
0      6   0.011507          0.0
1      9   0.002070          0.0
window==6 => p(0.0), conf(0.011506622516556292)
window==9 => p(0.0), conf(0.0020695364238410598)
Existen 1 grupos para ser añadidos con el primer valor comun
*********************************************
last window
window==6 => p(0.0),

In [74]:
contador = 1
for ley in consolidar.keys():
    print(consolidar.get(ley).toString())
    joblib.dump(consolidar.get(ley),'../objects/20newsgroup/rules_1_experiment/rule_1_'+str(contador))
    contador = contador+1

debería entrar aqui si no tengo antecedentes
OPERATOR IGUAL
(window==6 or 
debería entrar aqui si no tengo antecedentes
OPERATOR IGUAL
(window==6 or window==9 or 
(window==6 or window==9) => p(0.0), conf(0.013576158940397352)
debería entrar aqui si no tengo antecedentes
OPERATOR GREATER OR LOWER THAN
operator 1>=
((thank>=8 and thank<=10)) => p(0.0), conf(0.00024834437086092715)
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR GREATER OR LOWER THAN
operator 1>=
thank==4 and ((window>=2 and window<=3) or 
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR IGUAL window==5--window==5
thank==4 and ((window>=2 and window<=3) or window==5 or 
thank==4 and ((window>=2 and window<=3) or window==5 or 
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR IGUAL window==7--window==7
thank==4 and ((window>=2 and window<=3) or window==5 or window==7 or 
thank==4 and ((window>=2 and window<=3) or window==5 or window==7 or 
thank==4 and ((

In [ ]:
## 2 Experimento: Número de reglas 15, selección de vocabulario
vocabulary=['write','claim','person','nntp','article','university','window']
rs = RuleSet()
rs.target='target'
rs.pds=pds
rs.pds_original = pds 
explainer = Explainer(rs,1,10,15,200,vocabulary=vocabulary)
rs, pdsb = explainer.explain()


In [ ]:
rso = copy.deepcopy(rs)
for key in rso.dictionary.keys():
    print(rso.dictionary.get(key).toString())

In [ ]:
consolidar = rso.consolidar()

In [ ]:
contador = 1
for ley in consolidar.keys():
    print(consolidar.get(ley).toString())
    joblib.dump(consolidar.get(ley),'../objects/20newsgroup/rules_2_experiment/rule_2_'+str(contador))
    contador = contador+1

In [75]:
## 3 Experimento: Número de reglas 10, selección de vocabulario, número máximo de repeticiones por término 2

## 2 Experimento: Número de reglas 15, selección de vocabulario
vocabulary=['write','claim','person','nntp','article','university','window','computer','system','drive']
rs = RuleSet()
rs.target='target'
rs.pds=pds
rs.pds_original = pds 
explainer = Explainer(rs,1,5,10,3,vocabulary=vocabulary)
rs, pdsb = explainer.explain()

*****************************Comienza la genreación del ruleset************************************************
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: window
Igualdad generada: window==6POSITIVOS 139.0NEGATIVOS: 0
Igualdad de la regla window==6
-----------------> COUNTER USADOS
Counter({'window': 1})
|2 - Regla generada: |window==6 => p(0.0), conf(0.011506622516556292) 
ruleset query(window==6) 
	 |2.1 -Quedan | 11941 positivos, generadas 2 reglas
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: window
Igualdad generada: window==9POSITIVOS 25.0NEGATIVOS: 0
Igualdad de la regla window==9
-----------------> COUNTER USADOS
Counter({'window': 2})
|2 - Regla generada: |window==9 => p(0.0), conf(0.0020695364238410598) 
ruleset query(window==6) or (window==9) 
	 |2.1 -Quedan | 11916 positivos, generadas 3 reglas
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: window
Igualdad generada: window==10POSITIVOS 18.

In [76]:
rso = copy.deepcopy(rs)
for key in rso.dictionary.keys():
    print(rso.dictionary.get(key).toString())

window==6 => p(0.0), conf(0.011506622516556292)
window==9 => p(0.0), conf(0.0020695364238410598)
window==10 => p(0.0), conf(0.001490066225165563)
window==5 => p(0.0), conf(0.015728476821192054)
claim==1 and write==1 and person==1 and drive==7 => p(0.0), conf(0.0040562913907284765)
claim==1 and write==1 and person==1 and nntp==8 => p(0.0), conf(0.0008278145695364238)
claim==1 and write==1 and person==1 and nntp==6 and drive==1 => p(0.14285714285714285), conf(0.0028973509933774835)
claim==1 and write==1 and person==1 and drive==9 => p(0.0), conf(0.0018211920529801326)
nntp==6 and drive==1 and article==2 => p(0.0), conf(8.278145695364238e-05)
nntp==6 and article==1 and university==1 and computer==1 and system==1 => p(0.058823529411764705), conf(0.0014072847682119205)


In [77]:
consolidar = rso.consolidar()

0
diccionary
{0: <rulesets.Rule object at 0x7f1454999210>, 1: <rulesets.Rule object at 0x7f1454999390>, 2: <rulesets.Rule object at 0x7f1454999d50>, 3: <rulesets.Rule object at 0x7f1448e1b3d0>, 4: <rulesets.Rule object at 0x7f145a979450>, 5: <rulesets.Rule object at 0x7f1448df8690>, 6: <rulesets.Rule object at 0x7f1448f55d50>, 7: <rulesets.Rule object at 0x7f13116d83d0>, 8: <rulesets.Rule object at 0x7f13116d8450>, 9: <rulesets.Rule object at 0x7f131173b950>}
visitados q no deben volver :0 -- 0 -- 1 -- 2 -- 3
listado de reglas con mismo antecedente:
window==6 => p(0.0), conf(0.011506622516556292)
window==9 => p(0.0), conf(0.0020695364238410598)
window==10 => p(0.0), conf(0.001490066225165563)
window==5 => p(0.0), conf(0.015728476821192054)
listado_vari::
window
  window  confianza  consecuente
2     10   0.001490          0.0
3      5   0.015728          0.0
0      6   0.011507          0.0
1      9   0.002070          0.0
window==10 => p(0.0), conf(0.001490066225165563)
window==5 => p

In [78]:
contador = 1
for ley in consolidar.keys():
    print(consolidar.get(ley).toString())
    joblib.dump(consolidar.get(ley),'../objects/20newsgroup/rules_3_experiment/rule_3_'+str(contador))
    contador = contador+1

debería entrar aqui si no tengo antecedentes
OPERATOR GREATER OR LOWER THAN
operator 1>=
debería entrar aqui si no tengo antecedentes
OPERATOR GREATER OR LOWER THAN
operator 1>=
((window>=5 and window<=6) or (window>=9 and window<=10)) => p(0.0), conf(0.030794701986754967)
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR IGUAL drive==7--drive==7
claim==1 and write==1 and person==1 and (drive==7 or 
claim==1 and write==1 and person==1 and (drive==7 or 
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR IGUAL drive==9--drive==9
claim==1 and write==1 and person==1 and (drive==7 or drive==9 or 
claim==1 and write==1 and person==1 and (drive==7 or drive==9 or 
claim==1 and write==1 and person==1 and (drive==7 or drive==9) => p(0.0), conf(0.005877483443708609)
claim==1 and write==1 and person==1 and nntp==8 => p(0.0), conf(0.0008278145695364238)
claim==1 and write==1 and person==1 and nntp==6 and drive==1 => p(0.14285714285714285), conf(0.002897350

In [ ]:
h2o.init()

In [ ]:
modelo = h2o.upload_mojo('../objects/20newsgroup/modelo_xgb_2class/XGBoost_model_python_1594800800082_6894.zip')

In [ ]:
modelo.varimp()[]

In [ ]:
modelo2 = h2o.upload_mojo('../objects/20newsgroup/rf_model_2class/DRF_model_python_1594800800082_10408.zip')
modelo2.varimp()